### Resampling landsat
Resample the landsat 30 * 30 m data to the NOAA LAI resolution (0.05 * 0.05). 
We use the reproject_rematch from rioxarray which it performs the reprojection and then take the area based average. 


In [1]:
# prepare the libraries and data paths
import xarray as xr
import numpy as np
import rioxarray
from rasterio.enums import Resampling
import glob
import pandas as pd
import time

dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/"

in_dir = dir + ("data/processed_data/landsat/mosaic/")
out_dir = dir + "data/processed_data/landsat/resampled/"

target image is only a sample NOAA LAI

In [15]:
target_image = xr.open_dataset(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_monthly_resample_mean.nc"
)["LAI"].isel(time=5)
target_image = target_image.rio.write_crs("EPSG:4326").rename({"latitude":"y","longitude":"x"})

If out of memory use Dask

In [3]:
# Resampling TM5 data
for i in np.arange(1984, 2012):
    print("TM5 year:" + str(i))
    time.sleep(1)
    source_image = xr.open_dataarray(in_dir + "mosaic_NDVI_" + str(i) + "_TM5.tif",engine="rasterio")
    resampled = source_image.rio.reproject_match(
        target_image, resampling=Resampling.average
    )
    resampled.squeeze().to_netcdf(out_dir + "NDVI_resampled_" + str(i) + "_TM5.nc")
    del resampled

TM5 year:1984
TM5 year:1985
TM5 year:1986
TM5 year:1987
TM5 year:1988
TM5 year:1989


In [16]:
for i in np.arange(2014, 2015):
    print("ETM year:" + str(i))
    source_image = xr.open_dataarray(in_dir + "mosaic_NDVI_" + str(i) + "_ETM.tif",engine="rasterio")
    resampled = source_image.rio.reproject_match(
        target_image, resampling=Resampling.average
    )
    resampled.squeeze().to_netcdf(out_dir + "NDVI_resampled_" + str(i) + "_ETM.nc")
    del resampled

ETM year:2014


Here we stack all the files in one file for the trend analyses. Note that we use TM5 data from 1984-1999 and ETM from 1999-2014"

In [3]:
fnames = []
for i in np.arange(1984,2014):
    if i<1999:
        fnames.append(dir+"data/processed_data/landsat/resampled/NDVI_resampled_"+str(i)+"_TM5.nc")
        continue
    fnames.append(dir+"data/processed_data/landsat/resampled/NDVI_resampled_"+str(i)+"_ETM.nc")

# xr.open_dataset(fnames[0])["band_data"]
ds_ndvi = xr.concat([xr.open_dataset(f)["band_data"] for f in fnames],dim="time")    
ds_ndvi["time"] = pd.date_range("1984","2014",freq = "A")
ds_ndvi = ds_ndvi.drop("band").rename({"x":"longitude","y":"latitude"})
ds_ndvi.to_netcdf(dir+"data/processed_data/landsat/resampled/ndvi_annual_max.nc")